##### Copyright 2024 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemma - Game Design Brainstorming

This cookbook illustrates how you can employ the instruction-tuned model version of Gemma as a chatbot to aid in your game design process by fostering brainstorming sessions effectively.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/[Gemma_2]Game_Design_Brainstorming.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## Setup

### Select the Colab runtime
To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the Gemma model. In this case, you can use a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.


### Gemma setup on Kaggle
To complete this tutorial, you'll first need to complete the setup instructions at [Gemma setup](https://ai.google.dev/gemma/docs/setup). The Gemma setup instructions show you how to do the following:

* Get access to Gemma on kaggle.com.
* Select a Colab runtime with sufficient resources to run the Gemma 2B model.
* Generate and configure a Kaggle username and API key.

After you've completed the Gemma setup, move on to the next section, where you'll set environment variables for your Colab environment.

### Set environment variables

Set environment variables for `KAGGLE_USERNAME` and `KAGGLE_KEY`.

In [2]:
import os
os.environ["kaggle_username"] = "pavit0040604nis"


In [3]:
import os
from google.colab import userdata

os.environ["KERAS_BACKEND"] = "jax"  # Or "tensorflow" or "torch".

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["kaggle_username"] = "pavit0040604nis"
os.environ["kaggle_key"] = "fd2b0f46519b653d6ffe018d1dc1666d"

### Install dependencies

Install Keras and KerasNLP.

In [4]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras

## Core Concept

In [5]:
# A super concise description of your game
pitch = "In the heart of a reimagined Joseon Dynasty, where steam-powered machines and clockwork contraptions reign supreme, embark on an epic adventure to unravel the mysteries of a forgotten past and shape the destiny of a nation on the brink of revolution."

In [6]:
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle (2).json


In [10]:
import os
import shutil

# Define the target directory
kaggle_dir = "/root/.kaggle/"

# Create the directory if it doesn't exist
os.makedirs(kaggle_dir, exist_ok=True)

# Move kaggle.json to the correct location
shutil.move("kaggle.json", os.path.join(kaggle_dir, "kaggle.json"))

# Set the correct permissions
os.chmod(os.path.join(kaggle_dir, "kaggle.json"), 600)

print("✅ Kaggle API key set up successfully!")


Error: Destination path '/root/.kaggle/kaggle.json' already exists

## Load Model

In [7]:
import keras
import keras_nlp

# Run at half precision to fit in memory
keras.config.set_floatx("bfloat16")

import textwrap
from IPython.display import Markdown


def to_markdown(text):
    text = text.replace("•", "  *")
    return Markdown(textwrap.indent(text, "> ", predicate=lambda _: True))

gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_2b_en")
gemma_lm.compile(sampler="top_k")


class ChatState():
  """
  Manages the conversation history for a turn-based chatbot
  Follows the turn-based conversation guidelines for the Gemma family of models
  documented at https://ai.google.dev/gemma/docs/formatting
  """

  __START_TURN_USER__ = "<start_of_turn>user\n"
  __START_TURN_MODEL__ = "<start_of_turn>model\n"
  __END_TURN__ = "<end_of_turn>\n"

  def __init__(self, model, system=""):
    """
    Initializes the chat state.

    Args:
        model: The language model to use for generating responses.
        system: (Optional) System instructions or bot description.
    """
    self.model = model
    self.system = system
    self.history = []

  def add_to_history_as_user(self, message):
      """
      Adds a user message to the history with start/end turn markers.
      """
      self.history.append(self.__START_TURN_USER__ + message + self.__END_TURN__)

  def add_to_history_as_model(self, message):
      """
      Adds a model response to the history with start/end turn markers.
      """
      self.history.append(self.__START_TURN_MODEL__ + message)

  def get_history(self):
      """
      Returns the entire chat history as a single string.
      """
      return "".join([*self.history])

  def get_full_prompt(self):
    """
    Builds the prompt for the language model, including history and system description.
    """
    prompt = self.get_history() + self.__START_TURN_MODEL__
    if len(self.system)>0:
      prompt = self.system + "\n" + prompt
    return prompt

  def send_message(self, message):
    """
    Handles sending a user message and getting a model response.

    Args:
        message: The user's message.

    Returns:
        The model's response.
    """
    self.add_to_history_as_user(message)
    prompt = self.get_full_prompt()
    response = self.model.generate(prompt, max_length=4096)
    result = response.replace(prompt, "")  # Extract only the new response
    self.add_to_history_as_model(result)
    return result

  def show_history(self):
    for h in self.history:
      print(h)


chat = ChatState(gemma_lm)

## Elaborate with the given core game concept

In [8]:
result = chat.send_message(
    f"Elaborate about this game with the given core concept below.\n{pitch}"
)
to_markdown(result)

> ## A Clockwork Dynasty: Unveiling the Secrets of a Reimagined Joseon
> 
> This is a captivating concept for a game with a unique blend of historical and futuristic elements. Imagine a Joseon Dynasty, but instead of traditional bows and swords, it's powered by steam engines and intricate clockwork mechanisms. This world is on the cusp of a revolution, fueled by technological advancements and societal tensions, making for an intriguing narrative and gameplay experience. 
> 
> Here is a deeper dive into potential elements that could bring this game to life:
> 
> **Story & Setting:**
> 
> * **A World of Steam and Secrets:** The player is thrust into a world where traditional Joseon architecture is juxtaposed with towering clockwork factories, steam-powered airships soaring above ornate palaces, and intricate automata patrolling the streets. This creates a visually stunning environment with both historical charm and a futuristic edge. 
> * **The Whispers of the Past:** Ancient artifacts and hidden texts hold the key to unlocking the secrets of a forgotten past. The revolution is fueled by a desire to break free from an oppressive, technologically driven regime. The player must unravel the mysteries of this lost civilization to understand the true nature of the current power struggles.
> * **Choices that Shape Destiny:** The player's actions have a direct impact on the course of the nation. They can choose to support the revolutionaries or align with the current regime, each path leading to distinct consequences. Will the nation embrace progress or return to traditional values? 
> * **A Cast of Complicated Characters:** The player can encounter diverse characters, some loyal to the current regime, others seeking to dismantle it. They can be revolutionaries, inventors, merchants, or members of the aristocracy, all driven by their own motivations and desires.  
> * **The Clockwork Conspiracy:** The very heart of the game's conflict lies in the control of a powerful clockwork device, possibly a central source of energy or a weapon of mass destruction, hidden by the past. The player must navigate a web of intrigue and uncover its secrets to prevent a catastrophic outcome.
> 
>  **Gameplay:**
> 
> * **Strategic Combat:** Combines classic Joseon-style combat with steampunk weaponry like steam-powered gauntlets and automaton companions.  The player can strategically use environment, gadgets, and clockwork contraptions to gain the upper hand in battles. 
> * **Exploration and Discovery:** A vast open world filled with diverse environments, bustling cities, and hidden secrets.  Players can explore ancient ruins, delve into the depths of clockwork factories, and unravel the mysteries of the past. 
> * **Character Development:** Players can choose their character’s background, influencing their abilities, relationships, and overall playstyle. They can also customize their appearance and skills. 
> * **Crafting and Invention:** The player can build and modify their own clockwork contraptions, weapons, and even tools to aid them in their journey. 
> 
> 
> **Possible Genres:**
> 
> * **Action RPG:**  Combines fast-paced combat with exploration and character development. 
> * **Narrative-driven Adventure:**  Emphasizes storytelling and player choices, shaping the fate of the nation.
> * **Historical Fantasy:** Blends historical elements with steampunk elements for a unique setting. 
> 
> 
>  **Overall:**
> 
>  This game offers a rich and immersive experience, blending historical elements with futuristic technology. The core concept allows for compelling storytelling, strategic gameplay, and engaging exploration, making it a potential masterpiece for the gaming industry. The potential for a captivating narrative, intricate character development, and a diverse world of possibilities makes "A Clockwork Dynasty" a truly captivating concept. 
> <end_of_turn>

## Main Characters

In [9]:
result = chat.send_message("Design main characters.")
to_markdown(result)

> Here are some main character ideas that would fit within "A Clockwork Dynasty", embracing the blend of historical and futuristic elements:
> 
> **1.  The Renegade Inventor:  Han Seong-Jin**
> 
> * **Background:** An orphan who grew up on the streets, Han witnessed firsthand the injustices of the regime. Driven by a thirst for justice and fueled by a talent for clockwork engineering, he becomes a revolutionary leader.
> * **Personality:**  A charismatic and cunning leader with a quick wit and a knack for improvisation.  He believes in progress but knows the dangers of unchecked technology, often questioning the morality of the regime's ambitions.
> * **Special Ability:**  "Chronosmith": Can manipulate clockwork mechanisms to a surprising degree, creating powerful gadgets or augmenting his own body with mechanical enhancements.
> * **Visuals:** Lean and agile, often seen sporting goggles and a worn leather apron. His attire could be a mix of practical gear and salvaged clockwork parts for a unique aesthetic.
> 
> **2. The Noble Scholar: Kim So-Yeon**
> 
> * **Background:** A descendant of a prominent Joseon family, So-Yeon has always been fascinated by the ancient past and the forgotten knowledge hidden within it.  She finds herself drawn to the rebellion and the fight for a better future.
> * **Personality:**  A thoughtful and introspective character, deeply knowledgeable about history, but struggling to reconcile her family's legacy with the realities of the revolution. 
> * **Special Ability:**  "The Archivist": Can decipher and manipulate ancient texts, unlocking hidden knowledge and revealing secrets of the past. 
> * **Visuals:** Elegant and graceful, with flowing hair and delicate jewelry. She would be dressed in traditional court attire but with a touch of modernity, like a hidden pocket for her tools.
> 
> **3. The Shadowy Agent:  Park Jun-Seo**
> 
> * **Background:**  A former agent of the regime, Jun-Seo is disillusioned with the system's corruption and seeks to dismantle it from within. He is a master manipulator and strategist, adept at using technology to his advantage.
> * **Personality:**  Cold and calculating, driven by a thirst for power and a desire to reshape society. He is capable of great empathy and loyalty to those he sees as worthy.
> * **Special Ability:** "The Cipher": Possesses a vast network of contacts and access to secret information. He can manipulate events through subtle manipulation and leverage his connections.
> * **Visuals:** Darker complexion, sharp features, and a stoic expression. A sleek, dark outfit would be a contrast to his cold exterior.
> 
> **4. The Unconventional Warrior:  Lee Mi-Jin**
> 
> * **Background**: A talented warrior who has seen both the brutality of traditional warfare and the power of technology. She is fiercely independent and distrustful of both the regime and the revolution.
> * **Personality:** Strong-willed and pragmatic, she believes in strength and efficiency above sentiment. Her loyalty is earned, not given, and she's willing to fight for those she trusts.
> * **Special Ability:** "Automaton Warrior":  Master of a unique style of combat that combines traditional weapons with the precision and agility of clockwork automata, which she commands with incredible skill.  
> * **Visuals:**  A warrior's physique with scars to prove her battles. She wears practical, worn leather armor with intricate clockwork details. 
> 
> 
> **Additional Considerations:**
> 
> * **Interconnected Relationships:** These characters can have complex relationships, leading to alliances, betrayals, and personal journeys. Their individual stories will intertwine and influence the overall narrative. 
> * **Dynamic Choices:** The player's choices will affect their interactions with these characters, shaping the direction of the story.  
> 
> 
> By combining their unique personalities, skills, and backstories, "A Clockwork Dynasty" has potential to create a compelling cast of characters that will draw players into this exciting world. 
> <end_of_turn>

## Villain Characters

In [10]:
result = chat.send_message("Design villain characters.")
to_markdown(result)

> Here are some villain ideas, playing on the tension between tradition and progress in a Joseon Dynasty, reimagined through a steampunk lens:
> 
> **1.  Lord Kang Min-Woo, The Clockwork Emperor:**
> 
> * **Personality:** A charismatic, ruthless leader with a deep-seated belief in technological advancement above all else. He sees himself as a modern-day ruler, ushering in an era of unprecedented prosperity through relentless technological progress.
> * **Motivation:** Kang believes the old ways are flawed, holding humanity back. He seeks to establish a technologically controlled, almost godlike regime, using clockwork to enforce his will.
> * **Appearance:**  A tall, commanding figure, often dressed in ornate, metallic armor with intricate clockwork details. He exudes an air of both authority and a disturbing, almost mechanical, coldness.  
> 
> **2. Dr. Choi Jae-Hwan: The Clockwork Alchemist**
> 
> * **Personality:** A brilliant but twisted scientist obsessed with achieving immortality and control through the manipulation of ancient, forbidden magic and clockwork. He sees technology as his tool for achieving this goal.
> * **Motivation:** Jae-Hwan believes he is the chosen one, destined to unlock the secrets of the world through his clockwork creations, and sees himself as a god-like figure who will shape the future.
> * **Appearance:**  An eccentric and gaunt man, with wild, unkempt hair and piercing blue eyes. He wears strange, elaborate robes adorned with clockwork gears and glowing crystals. 
> 
> **3.  The Iron Guard:  The Secret Society of Clockwork Soldiers**
> 
> * **Background:** An elite military organization loyal only to Lord Kang, they are a fearsome group of cyborgs, automatons, and clockwork-enhanced soldiers. 
> * **Motivation:** The Iron Guard is a symbol of Kang's authority, their unwavering obedience a testament to their devotion to him and their ruthless pursuit of his vision.
> * **Appearance:**  Their uniforms are sleek and imposing, incorporating clockwork mechanisms that amplify their strength and agility. Their weaponry is a mix of traditional weapons and advanced, clockwork-driven tools. 
> 
>  **4.  The Clockwork Queen:  Lady Seo Jin-Hee**
> 
> * **Personality:** A cunning, manipulative woman who uses her influence and beauty to manipulate the court and advance her own ambitions. She is deeply suspicious and distrustful of outsiders, including the revolutionaries, and seeks to control them.
> * **Motivation:** Seo is obsessed with preserving the status quo, believing that tradition is the only true path towards stability. She is a skilled manipulator who sees herself as a necessary counterpoint to the revolutionaries' chaotic ambition.
> * **Appearance:**  An elegant woman with a striking presence. She wears luxurious clothing and jewels, and often utilizes her charm and beauty to influence others, while hiding a cold, calculating mind.
> 
> 
> **Additional Notes:**
> 
> * **Conflict:** These villains can represent different factions within the conflict. Lord Kang might represent the regime's power, while Dr. Choi represents the more sinister aspects of technological advancement.  The Iron Guard could be a symbol of the regime's military might, while Lady Seo embodies the resistance's more subtle and manipulative tactics.
> * **Moral Ambiguity:**  Even these villainous figures might have moments of humanity or moments where their actions are motivated by a sense of justice, even if it is twisted. 
> * **Player Agency:** The player's choices can influence the actions of these villains.  Will the player choose to confront them directly or seek a different path to dismantle their power? 
> 
> 
> 
> 
> These villains can add depth and intrigue to the "A Clockwork Dynasty" world. By creating a diverse cast of antagonists, you can provide compelling motivations and opportunities for engaging gameplay. 
> 
> 
> <end_of_turn>

## Main Story

In [11]:
result = chat.send_message(
    "Write the main story of this game with an introduction, development, turn, and conclusion."
)
to_markdown(result)

> ## A Clockwork Dynasty: The Whispers of Revolution
> 
> **Introduction:** 
> 
> Seoul, 1888. Steam-powered airships drift through the clouds, while clockwork automata patrol the streets. The Joseon Dynasty, once steeped in tradition, now teeters on the edge of a technological revolution. The city hums with innovation, but underneath, tensions are rising as the elite, fueled by a desire for progress at any cost, clash with the oppressed masses seeking a return to a simpler, more equitable world.
> 
> You are Han Seong-Jin, a young orphan who witnessed the brutal crackdown on the first rebellion.  You've honed your skills as a clockwork inventor and leader, driven by a burning desire to dismantle the regime's oppressive power.  
> 
> **Development:** 
> 
> The game begins with a clandestine meeting in a hidden clockwork workshop. You are introduced to other characters:  So-Yeon, the scholarly descendant of a noble family, and Jun-Seo, a former regime agent turned rebel. They share your vision of a better future, but their paths diverge: So-Yeon seeks knowledge to understand the past, Jun-Seo seeks power to dismantle the current regime. 
> 
> As you navigate the city's underbelly, you encounter the Iron Guard, Kang Min-Woo's elite military force, who enforce his iron fist. The regime’s control is suffocating, and the people yearn for freedom. You are thrust into a world of political intrigue, daring escapes, and dangerous encounters with both the regime's forces and the revolutionaries. 
> 
> **The First Turn:  The Clockwork Conspiracy:**
> 
> You uncover a secret conspiracy within the Iron Guard – a plan to unleash a devastating weapon powered by ancient, forbidden magic and clockwork technology.  Kang Min-Woo, the Clockwork Emperor, is the orchestrator.  The game's central conflict begins to take shape.
> 
> You are forced to choose a side, and the consequences are profound.  Do you align yourself with Jun-Seo and his clandestine rebel network, or with So-Yeon and her pursuit of knowledge? Your choices will influence the direction of the revolution, the fate of the regime, and your own path. 
> 
> **The Rising Tension**
> 
> The conflict intensifies as you uncover the secrets of the Clockwork Emperor's weapon.  The Iron Guard becomes more ruthless in their pursuit of control. You must race against time to expose Kang’s plot to dismantle his power, but the stakes are high.  
> 
> You face difficult choices.  Will you trust your allies and fight alongside them? Or will you choose solitude and independence, risking the potential loss of those you hold dear? Your decisions will impact the future of the nation and your own personal journey.
> 
> **The Turning Point:**
> 
> A pivotal moment arrives: a clash with the Iron Guard, a daring heist to acquire crucial data, or a confrontation with Kang Min-Woo himself.  The player's choices will determine their impact on the revolution, shaping the narrative and leading to a decisive outcome.
> 
> Will the revolution succeed? Will the Clockwork Emperor be brought to justice? Will the nation be reborn in the image of its past or its future? 
> 
> 
> **Conclusion:**
> 
> The final chapter of the game sees the culmination of the player’s choices and the consequences that followed.  The fate of the Joseon Dynasty is in the balance, and the outcome will determine the legacy of the revolution.
> 
> * **A New Era:** The ending can be bittersweet, with hope for a new dawn, but also with the understanding of the challenges that lie ahead in the aftermath of a tumultuous revolution, where the past and the future must coexist.
> *  **A Cycle of Power:** The ending can also be bleak, with power shifting, leaving the future uncertain. The Clockwork Dynasty might be reborn in a more brutal form, leaving a lingering threat of tyranny.
> * **Hope for a New World:**  A hopeful ending might involve a new system of governance, where tradition and technology coexist in harmony. The player's choices shape a new future. 
> 
> 
> 
> 
>  **Key Themes to Explore:** 
>  *  The balance between tradition and progress.
>  *  The power and corruption of technology.
>  *  The fight for freedom and the cost of revolution. 
>  *  The complexities of loyalty and the choices we make in the face of adversity. 
>  * The importance of individual agency against overwhelming power structures.
> 
> 
> 
> 
> This outline provides a foundation for your game’s narrative, allowing for player agency and diverse outcomes. It's essential to weave together compelling characters, intriguing plot twists, and thought-provoking themes to create a compelling and unforgettable gaming experience. 
> <end_of_turn>

## Show the chat history

In [12]:
chat.show_history()

<start_of_turn>user
Elaborate about this game with the given core concept below.
In the heart of a reimagined Joseon Dynasty, where steam-powered machines and clockwork contraptions reign supreme, embark on an epic adventure to unravel the mysteries of a forgotten past and shape the destiny of a nation on the brink of revolution.<end_of_turn>

<start_of_turn>model
## A Clockwork Dynasty: Unveiling the Secrets of a Reimagined Joseon

This is a captivating concept for a game with a unique blend of historical and futuristic elements. Imagine a Joseon Dynasty, but instead of traditional bows and swords, it's powered by steam engines and intricate clockwork mechanisms. This world is on the cusp of a revolution, fueled by technological advancements and societal tensions, making for an intriguing narrative and gameplay experience. 

Here is a deeper dive into potential elements that could bring this game to life:

**Story & Setting:**

* **A World of Steam and Secrets:** The player is thrust 

## Clear History

In [13]:
chat.history.clear()

## Expansion Idea

### Marketing Phrase

In [14]:
chat.history.clear()
pitch = "A new steam-powered toothbrush"
result = chat.send_message(
    f"Generate a marketing phrase for the new product below.\n{pitch}"
)
to_markdown(result)

> Here are some marketing phrases for a new steam-powered toothbrush, playing on different angles:
> 
> **Focusing on the Technology:**
> 
> * **Steam-Powered Clean: Experience the future of oral care.**
> * **Revolutionize Your Brushing with Revolutionary Steam Technology.**
> * **The Power of Steam: Deep clean, fresh breath, every time.**
> * **Unleash the Steam Cleaning Power for a Brighter Smile.**
> 
> **Highlighting Benefits:**
> 
> * **Go Beyond Traditional Brushing: Steam-Powered Cleaning for Deeper Clean.**
> * **The Ultimate Clean, Effortlessly: Steam Technology at Your Fingertips.**
> * **Say Goodbye to Plaque, Hello to Fresh Breath: Experience Steam Cleaning.**
> * **Feel the Difference: Steam-Powered Brushing for a Healthier Smile.**
> 
> **Creating a Brand Story:**
> 
> * **Introducing [Brand Name]: The Steam Powered Revolution in Oral Care.**
> * **A Breath of Fresh Air: Discover the Power of Steam in Oral Care.**
> * **Forging a Path to a Brighter Smile: Embrace Steam Technology.**
> * **The Future of Oral Care is Here: Introducing the Steam-Powered Toothbrush.**
> 
> **Catchy and Short:**
> 
> * **Steam Clean, Shine Bright.**
> * **The Next Generation of Oral Care.**
> * **Unleash the Steam.**
> * **Fresh Breath, Steam Power.**
> 
> 
>  **Tips for choosing the best phrase:**
> 
> * **Consider your target audience.** Who are you trying to reach? 
> * **Focus on the key benefits of the product.** 
> * **Use strong and memorable language.**
> * **Keep it short and easy to remember.** 
> * **Test different phrases and see which ones resonate best.**
> 
> 
> 
> Let me know if you need more options!<end_of_turn>

### Florist Ideas

In [15]:
chat.history.clear()
pitch = "Universe and shooting stars"
result = chat.send_message(
    f"Generate a florist idea inspired by the concept below, along with suggestions for suitable flowers.\n{pitch}"
)
to_markdown(result)

> ## Florist Idea: "Celestial Blooms"
> 
> **Concept:** This florist would capture the beauty and magic of the universe and the thrill of a shooting star, using celestial elements in their designs. Think starry skies, cosmic swirls, and the awe-inspiring wonder of space.
> 
> **Theme:**
> 
> * **Color Palette:** Deep blues, blacks, purples, silver, gold
> * **Decor:**  Starry night sky fabric draping, constellations on shelves, hanging moon phases, glow-in-the-dark elements. 
> * **Floral Styles:**
>     * **Starry Arrangements:** Tall, elegant arrangements with long-stemmed flowers like orchids, calla lilies, or protea in shades of purple, blue, or silver, mimicking shooting stars shooting through the sky. 
>     * **Galaxy Bowls:**  Deep, round bowls with cascading flowers like hydrangeas, roses, and lilies, creating a galaxy effect. 
>     * **Moon and Sun:**  A single large sunflower or daisy surrounded by smaller blooms in moon and star shapes.
>     * **Constellation Arrangements:**  Flowers arranged in a stylized constellation pattern, using different colored foliage and textures.
> 
> **Suitable Flowers:**
> 
> * **Stargazer Lilies:** Their unique star-shaped blooms are perfect for capturing the shooting stars theme.
>     
> * **Calla Lilies:** Elegant and graceful with a variety of colors, they create a sense of mystery.
> 
> * **Orchids:**  Long-lasting and available in a range of colors, orchids add a touch of sophistication to the arrangements.
> 
> * **Hydrangeas:**  These blooms create a sense of fullness and can be used to form galaxy shapes.
> 
> * **Roses:**  In shades of purple, blue, white, and silver, roses add romance and elegance.
> 
> * **Proteas:**  These unique flowers with their bold colors and textures create a striking effect.
> 
> **Additional Touches:**  
> 
> * Offer custom arrangements with specific constellations.
> * Use celestial maps and star charts as decorative elements.
> * Include small, glowing crystals, like moonstones or amethyst, as accent pieces.
> * Offer a selection of celestial-themed candles or incense. 
> 
> **Target Audience:** This florist would appeal to people who love the beauty of nature, space exploration enthusiasts, and anyone looking for a unique and memorable floral experience.  
> 
> This florist idea offers endless possibilities to create a truly magical and inspiring space for customers to experience. 
> <end_of_turn>

### Food Recipe

In [16]:
chat.history.clear()
pitch = "Cyberpunk Kraken"
result = chat.send_message(
    f"Generate a cooking recipe with the concept below.\n{pitch}"
)
to_markdown(result)

> ## Cyberpunk Kraken: Spiced Squid Ink Pasta
> 
> **Inspired by the dark depths of the megacity's underbelly and the elusive Kraken, this dish is a fusion of cyberpunk grit and seafood delicacy.**
> 
> 
> **Concept:**
> 
> * **Color & Texture:** Deep black squid ink pasta, reminiscent of a shadowy abyss. 
> * **Flavor:** Spicy, umami, and smoky, evoking a cyberpunk urban landscape.
> * **Texture:**  A mix of chewy squid ink noodles, crisp vegetables and a rich, spicy sauce.
> 
> **Ingredients:**
> 
> * **Pasta:** 12 oz Squid Ink Pasta (find it in specialty gourmet stores, often sold as "black pasta" or "squid ink pasta")
> * **Squid:** 1 lb Fresh, cleaned squid tentacles, cut into bite-sized pieces (can substitute with frozen calamari)
> * **Vegetables:**  
>     * 1 large onion, diced
>     * 2 cloves garlic, minced
>     * ½ red bell pepper, diced
>     * 1 cup chopped mushrooms (cremini or portobello work best) 
> * **Spice:** 
>     * 1 tbsp Smoked paprika
>     * 1 tsp Chili flakes (adjust to your spice preference)
>     * 1 tsp ground black pepper
>     * 1/2 tsp cumin powder
>     * Salt to taste
>     * Pinch of cayenne pepper (optional, for extra heat) 
> * **Sauce:** 
>     * 2 tbsp olive oil
>     * 1/4 cup vegetable broth
>     * 2 tbsp tomato paste
>     * 1/4 cup dry red wine (optional, but adds complexity)
>     * 1/4 cup heavy cream (optional, for richness)
> * **Garnish:** 
>     * Fresh chopped parsley 
>     * Lemon wedges
>     * Crumbled feta cheese (optional, for a salty kick)
>     * Microgreens (optional, for a futuristic flair)
> 
> **Instructions:** 
> 
> 1. **Cook the Pasta:** Bring a large pot of salted water to a rolling boil.  Cook the squid ink pasta according to package directions, until it reaches al dente.  
> 2. **Sear the Squid:** Heat 2 tbsp of olive oil in a large pan over medium-high heat.  Sear the squid tentacles, cooking for 2-3 minutes on each side until they are lightly browned. Remove the squid from the pan and set aside.
> 3. **Sauté the Vegetables:** Add the onion and garlic to the same pan and sauté until softened, about 5 minutes.  Add the diced red pepper, mushrooms, and spices. Sauté for another 5 minutes, stirring occasionally. 
> 4. **Create the Sauce:**  Stir in the vegetable broth, tomato paste, and red wine (if using). Bring to a simmer and cook for 5-7 minutes, or until the sauce has thickened slightly.  
> 5. **Combine & Add the Squid:**  Add the cooked squid tentacles and the heavy cream (if using) to the pan and stir to coat. 
> 6. **Serve:** Drain the squid ink pasta and add it to the pan with the sauce. Toss to combine and cook for 1-2 minutes, ensuring the sauce coats the pasta evenly. Garnish with chopped parsley, a lemon wedge, crumbled feta cheese (optional), and microgreens (optional).
> 
>  **Presentation:**  
> 
> * Serve the Cyberpunk Kraken in a dark ceramic bowl or on individual plates with a cyberpunk-style garnish like a stylized LED light strip or neon paint splatters.  
> *  Use chopsticks or a futuristic fork for a cyberpunk aesthetic.
> 
> 
> **Cyberpunk Inspiration:**
> 
> *  The deep black ink pasta and squid tentacles create a visual connection to the shadowy depths of the megacity.
> * The spicy and smoky flavors reflect the gritty and rebellious underbelly of the city.
> *  The optional garnishes add a futuristic flair and enhance the cyberpunk theme.
> 
> **Enjoy your delicious and cyberpunk-inspired dish!** 
> <end_of_turn>